In [1]:
# 4章 ニューラルネットワークの学習
# 4.2 損失関数
# 損失関数はニューラルネットワークの性能の「悪さ」を示す指標
#  現在のニューラルネットワークが教師データに対してどれだけ適合していないか、教師データに対してどれだけ一致していないかということを表す

In [2]:
# 4.2.1 2乗和誤差

# この配列の要素は、最初のインデックスから順に、数字の「０」、「１」、「２」・・に対応

# ここでニューラルネットワークの出力であるyは、ソフトマックス関数の出力
#  「０」の確率は0.1、「１」の確率は0.05、「２」の確率は0.6といったようなことを表す
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

# tは教師データ
# この教師データは正解となるラベルを1、それ以外を0とする
#  ここではラベルの「２」が1なので、正解は「２」であることを表す
#  なお、正解ラベルを1として、それ以外は0で表す表記法をone-hot表現という
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [3]:
# 2乗和誤差はニューラルネットワークの出力と正解となる教師データの各要素の差の2乗を計算し、その総和を求める
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [4]:
# 「２」を正解とする
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [5]:
# 例１：「２」の確率が最も高い場合(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

In [6]:
import numpy as np
mean_squared_error(np.array(y), np.array(t))

0.09750000000000003

In [7]:
# 例２：「７」の確率が最も高い場合(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

In [8]:
mean_squared_error(np.array(y), np.array(t))

0.5975

In [9]:
# この実験の結果で示されるように、一つ目の例の損失関数のほうが小さくなっており、教師データとの誤差が小さいことがわかる
#  つまり、一つ目の例の方が、出力結果が教師データにより適合していることを2乗和誤差は示している

In [10]:
# 4.2.2 交差エントロピー誤差

In [13]:
# 交差エントロピー誤差を実装
def cross_entropy_error(y, t):
    # np.log(0)のような計算が発生した場合、np.log(0)はマイナスの無限大を表す-infとなり、
    #  そうなってしまうと、それ以上計算を進めることが出来なくなる
    # その防止策として、微小な値を追加して、マイナス無限大を発生させないようにしている
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [14]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [16]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

In [17]:
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [18]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

In [19]:
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

In [20]:
# 4.2.3 ミニバッチ学習
# ビックデータともなると、すべてのデータを対象とした損失関数を計算するのは現実的ではない
#  そこで、データの中から一部を選び出し、その一部のデータを全体の「近似」として利用するーこれをミニバッチ(小さな塊)というー
# そのミニバッチごとに学習する。たとえば、60000枚の訓練データの中から100枚を無作為に選びだして、その100枚を使って学習を行う
#  このような学習方法をミニバッチ学習という

In [21]:
import sys, os
os.chdir('C:\\Users\\200286\\workspace\\github\\gitlocalrep\\deep-learning-from-scratch\\ch03') # カレントディレクトリをch03に変更
sys.path.append(os.pardir) # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from dataset.mnist import load_mnist

In [22]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [23]:
print(x_train.shape)

(60000, 784)


In [25]:
print(t_train.shape)

(60000, 10)


In [27]:
# この訓練データの中からランダムに10枚だけ抜き出す
train_size = x_train.shape[0]
batch_size = 10

# np.random.choice()を使えば、指定された数字の中からランダムに好きな数だけ取り出すことができる
#  たとえば、np.random.choice(60000, 10)とすると、0から60000未満の数字の中からランダムに10個の数字を選び出す
batch_mask = np.random.choice(train_size, batch_size)

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [28]:
np.random.choice(60000, 10)

array([30995, 59913,  1785, 36578, 31537,  5361, 53045,  4928,  5398,
       58076])

In [29]:
# 後は、このランダムに選ばれたインデックスを指定して、ミニバッチを取り出すだけ。このミニバッチを使って、損失関数を計算する

In [30]:
# 4.2.4 [バッチ対応版]交差エントロピー誤差の実装
# 先ほど実装した交差エントロピー誤差は一つのデータを対象とした誤差なので、それを改良する

# yはニューラルネットワークの出力、tは教師データとする
def cross_entropy_error(y , t):
    
    # yの次元が1の場合、つまり、データひとつあたりの交差エントロピー誤差を求める場合は、データの形状を整形する
    if y.sum == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size